In [ ]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup
import marqo
import pprint

In [ ]:
df = pd.read_csv('./data/filevine_case_study_justice - train set_source data set.csv')
model = "hf/multilingual-e5-large"
url = "http://localhost:8882"
mq = marqo.Client(url=url)

In [ ]:
df = pd.read_csv('./data/filevine_case_study_justice - train set_source data set.csv')

def find_tags(txt):
    soup = BeautifulSoup(txt, 'html.parser')
    tags =[tag.name for tag in soup.find_all()]
    return tags

def clean_text(txt):
    soup = BeautifulSoup(txt.strip(), 'html.parser')
    clean_text = soup.get_text()
    clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)
    clean_text = clean_text.lower()
    return clean_text

df['facts_clean'] = df['facts'].apply(clean_text)

df['sim1_vector'] = None
df['sim1_score'] = None
df['sim1_href'] = None
df['sim2_vector'] = None
df['sim2_score'] = None
df['sim2_href'] = None
df['sim3_vector'] = None
df['sim3_score'] = None
df['sim3_href'] = None

df['sim1_vector_sent'] = None
df['sim1_score_sent'] = None
df['sim1_href_sent'] = None
df['sim2_vector_sent'] = None
df['sim2_score_sent'] = None
df['sim2_href_sent'] = None
df['sim3_vector_sent'] = None
df['sim3_score_sent'] = None
df['sim3_href_sent'] = None

In [ ]:
for index, row in df.iterrows():
    search_href = row['href']
    facts = row['facts']
    results = mq.index("filevine_docs_whole").search(facts, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df[0:3]

    df.loc[df['href']==search_href, 'sim1_score']=results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim1_href'] = results_df['href'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_score'] = results_df['_score'].iloc[1]
    df.loc[df['href']==search_href, 'sim2_href'] = results_df['href'].iloc[1]
    df.loc[df['href']==search_href, 'sim3_score'] = results_df['_score'].iloc[2]
    df.loc[df['href']==search_href, 'sim3_href'] = results_df['href'].iloc[2]

    for index, row in results_df[0:3].iterrows():
        document_id = row['_id']
        facets = mq.index("filevine_docs_whole").get_document(
            document_id=document_id,
            expose_facets=True
        )
        
        df.loc[df['href']==row['href'], f'sim{index}_vector'] = str(facets['_tensor_facets'][0]['_embedding'])

In [ ]:
for index, row in df.iterrows():
    search_href = row['href']
    facts = row['facts']
    results = mq.index("filevine_docs_sentences").search(facts, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df[0:3]

    df.loc[df['href']==search_href, 'sim1_score_sent']=results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim1_href_sent'] = results_df['href'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_score_sent'] = results_df['_score'].iloc[1]
    df.loc[df['href']==search_href, 'sim2_href_sent'] = results_df['href'].iloc[1]
    df.loc[df['href']==search_href, 'sim3_score_sent'] = results_df['_score'].iloc[2]
    df.loc[df['href']==search_href, 'sim3_href_sent'] = results_df['href'].iloc[2]

    for index, row in results_df[0:3].iterrows():
        document_id = row['_id']
        facets = mq.index("filevine_docs_sentences").get_document(
            document_id=document_id,
            expose_facets=True
        )

        df.loc[df['href']==row['href'], f'sim{index}_vector_sent'] = str(facets['_tensor_facets'][0]['_embedding'])

In [ ]:
df.to_csv('embedding_df.csv')